# Interactive time-gating

## Preliminaries

### Imports

In [ ]:
%matplotlib inline
import numpy as np
import copy
import interactive_gating_with_unc_utils as utils
base = utils.BaseMethods()

### Compare different available datasets

In [ ]:
base.compare_different_datasets()

## Time Gating

### Load Dataset

In [ ]:
# data to be used for further processing
data_raw = f, s11_ri, s11_ri_cov = base.load_data("empirical_cov")

# get corresponding time
Nx = len(s11_ri) - 1
t_span = 1 / np.mean(np.diff(f))  # original f, not f_mod
t = np.linspace(0, t_span, num=Nx)

### Time Gating Process Settings

In [ ]:
# define window and gate
w, uw = base.window(size=len(f), kind="neutral")
gate = lambda t: base.gate(t, t_start=0.0, t_end=0.18, kind="kaiser", order=2.5*np.pi)

# store settings in dicts
data = {"f": f, "s_ri": s11_ri, "s_ri_cov": s11_ri_cov}
config = {
    "window": {"val": w, "cov": uw},
    "zeropad": {"pad_len": 2500, "Nx": Nx},
    "gate": {"gate_func": gate, "time": t},
    "renormalization": None,
}

### Perform the Gating Using Two Different Approaches

#### Method 1

Direct Evaluation and Elementwise Multiplication in Time-Domain

In [ ]:
result_m1 = base.perform_time_gating_method_1(data, config, return_internal_data=True)

#### Method 2

Monte Carlo and Complex Convolution in Frequency-Domain

In [ ]:
result_m2 = base.perform_time_gating_method_2(data, config)

## Visualiziations

In [ ]:
# settings to obtain raw signal in the time domain
config_nomod = copy.deepcopy(config)
config_nomod["window"] = None
config_nomod["zeropad"] = None
result_nomod = base.perform_time_gating_method_1(data, config_nomod, return_internal_data=True)

### Compare Results of Different Methods to Original S-Parameter Spectrum

In [ ]:
args_raw = {"l": "original", "c": "tab:gray"}
args_m1 = {"l": "gated (method 1)", "c": "tab:blue", "lw": 5}
args_m2 = {"l": "gated (method 2)", "c": "tab:orange"}

plotdata_comparison = [
    [data_raw, args_raw],
    [tuple(result_m1["data"].values()), args_m1],
    [tuple(result_m2["data"].values()), args_m2],
]

cs_mag_phase = {
    3: {
        "ylim": (1e-3, 1.5e1),
    }
}

In [ ]:
base.mag_phase_plot(plotdata_comparison, custom_style=cs_mag_phase)

### Visualize S-Parameter and Gate in the Time-Domain

In [ ]:
args_raw = {"l": "raw", "c": "tab:gray"}
args_mod = {"l": "modified", "c": "tab:green", "lw": 2}
args_gate = {"l": "gate", "c": "red"}

plotdata_timedomain = [
    [tuple(result_nomod["internal"]["modified"].values()), args_raw],
    [tuple(result_m1["internal"]["modified"].values()), args_mod],
    [tuple(result_m1["internal"]["gate"].values()), args_gate],
]

cs_timedomain = {
    0: {
        "xlim": (-0.1, 0.65),
    },
    1: {
        "yscale": "linear",
    }
}

In [ ]:
base.time_domain_plot(plotdata_timedomain[:2], custom_style=None)

In [ ]:
base.time_domain_plot(
    plotdata_timedomain, custom_style=cs_timedomain, last_dataset_has_own_axis=True
)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
if False:
    array_export = np.c_[t_shifted_mod, S11_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod))]
    df_export = pandas.DataFrame(array_export)
    df_export.columns = ["time", "signal", "signal_unc"]
    df_export.to_excel("export.xlsx")

In [ ]:
# visualize time domain data
fig_in_td_cov, ax_in_td_cov = plt.subplots(nrows=1, figsize=(6, 6), tight_layout=True)
maxi = np.max(np.abs(S11_cov))
cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=1e-14)

img0 = ax_in_td_cov.imshow(S11_cov_shifted_mod,  cmap="PuOr", norm=cnorm)

fig_in_td_cov.colorbar(img0, ax=ax_in_td_cov)
ax_in_td_cov.set_title("Covariance of (modified) time signal")

### Covariance Matrices

Covariances in Re/Im Representation of:

- input data
- output of method 1 (analytical uncertainty evaluation)
- output of method 2 (Monte Carlo uncertainty evaluation)

In [ ]:
# utility function for annotating plots
def annotate_real_imag_plot(ax, annotate_x=True, annotate_y=True):
    # define new tick positions
    labels_re = [0, 10, 20, 30]  # GHz
    labels_re = [0, 8, 16, 24]  # GHz
    labels_im = labels_re
    labels = labels_re + labels_im

    # define new labels for these positions
    ticks_re = [np.flatnonzero(f == l).item() for l in labels_re]
    ticks_im = [f.size + 1 + k for k in ticks_re]
    ticks = ticks_re + ticks_im

    # define colors for the labels to distinguish real and imag parts
    colors_re = ["k"] * len(ticks_re)
    colors_im = ["r"] * len(ticks_im)
    tick_colors = colors_re + colors_im

    if annotate_x:
        # xticks (label, position, color)
        ax.set_xticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_xticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_xlabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(0.25, -0.01),
            xytext=(0.25, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(0.75, -0.01),
            xytext=(0.75, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    if annotate_y:
        # yticks (label, position, color)
        ax.set_yticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_yticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_ylabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(-0.01, 0.75),
            xytext=(-0.10, 0.75),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(-0.01, 0.25),
            xytext=(-0.10, 0.25),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    return ax


In [ ]:
fig_cov, ax_cov = plt.subplots(nrows=3, figsize=(8, 25), tight_layout=True)
mini = max(1e-11, min(np.abs(s11_ri_cov).min(), np.abs(s11_gated_ri_cov).min()))
maxi = min(np.abs(s11_ri_cov).max(), np.abs(s11_gated_ri_cov).max())

cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=mini)
img1 = ax_cov[0].imshow(s11_ri_cov, cmap="PuOr", norm=cnorm)
img2 = ax_cov[1].imshow(s11_gated_ri_cov, cmap="PuOr", norm=cnorm)
img3 = ax_cov[2].imshow(s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_cov.colorbar(img1, ax=ax_cov[0])
fig_cov.colorbar(img2, ax=ax_cov[1])
fig_cov.colorbar(img3, ax=ax_cov[2])

ax_cov[0].set_title("Covariance of s11_ri")
ax_cov[0] = annotate_real_imag_plot(ax_cov[0])

ax_cov[1].set_title("Covariance of s11_gated_ri")
ax_cov[1] = annotate_real_imag_plot(ax_cov[1])

ax_cov[2].set_title("Covariance of s11_gated_mcconv_ri_cov")
ax_cov[2] = annotate_real_imag_plot(ax_cov[2])

#### Compare Gated Spectra and Covariance Matrices of analytical and Monte Carlo approach

In [ ]:
# visual comparison of the two methods:
fig_comp, ax_comp = plt.subplots(nrows=3, figsize=(8, 20), tight_layout=True)

# mean signed difference of values
ax_comp[0].plot(s11_gated_ri - s11_gated_mcconv_ri)
ax_comp[0].set_title("Mean Signed Difference of Gated Spectra")
ax_comp[0] = annotate_real_imag_plot(ax_comp[0], annotate_y=False)

# mean signed difference of covariance matrices
img4 = ax_comp[1].imshow(s11_gated_ri_cov - s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_comp.colorbar(img4, ax=ax_comp[1])

ax_comp[1].set_title("Signed Difference of both Covariance Matrices")
ax_comp[1] = annotate_real_imag_plot(ax_comp[1])

# Kullback-Leibler divergence of covariance matrices
kl_div = special.kl_div(s11_gated_ri_cov, s11_gated_mcconv_ri_cov)
cnorm_kl = colors.LogNorm(vmin=1e-12, vmax=1e-8, clip=True)

img5 = ax_comp[2].imshow(kl_div, cmap="binary", norm=cnorm_kl)
fig_comp.colorbar(img5, ax=ax_comp[2])


ax_comp[2].set_title("Kullback-Leibler divergence of both Covariance Matrices")
ax_comp[2] = annotate_real_imag_plot(ax_comp[2])